## Seleções &amp; Ligações, Revisitados

In [2]:
import pandas as pd
import altair as alt

cars = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/cars.json'
movies = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/movies.json'
sp500 = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/sp500.csv'
stocks = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/stocks.csv'
flights = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/flights-5k.json'

Anteriormente neste trabalho, vimos um exemplo de _seleções &amp; ligações_: usando um histograma de consulta dinâmica para destacar pontos em um gráfico de dispersão de classificação de filmes. Aqui, visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Voltando ao conjunto de dados `cars`, podemos utilizar o operador `repeat` para construir uma [matriz de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência. Podemos definir uma seleção `intervalo` e incluí-la _dentro_ da nossa especificação de gráfico de dispersão repetido para permitir seleções associadas entre todos os gráficos.

_Clique e arraste em qualquer um dos gráficos abaixo para executar a seleção &amp; ligação!_

In [ ]:
brush = alt.selection_interval(
    resolve='global' # resolver todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

C:\Users\elian\AppData\Local\Temp\ipykernel_19272\3480946980.py:5: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  alt.Chart(cars).mark_circle().add_selection(


alt.RepeatChart(...)

Note acima a utilização de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que em todos os gráficos apenas uma seleção pode estar ativa por vez. No entanto, em alguns casos, podemos querer definir seleções em múltiplos gráficos e combinar os resultados. Se utilizarmos `resolve='union'`, a seleção será a _união_ de todos as seleções: se um ponto residir em qualquer uma delas, ele será selecionado. Alternativamente, se utilizarmos `resolve='intersect'`, a seleção consistirá na _interseção_ de todos as seleções: apenas os pontos que residem dentro de todas as seleções serão selecionados.

_Tente definir o parâmetro `resolve` como `union` e `intersect` e veja como isso muda a lógica de seleção resultante._

### Filtragem cruzada

Todos os exemplos de seleção &amp; ligação que vimos utilizam codificações condicionais, por exemplo, para alterar os valores de opacidade em resposta a uma seleção. Outra opção é usar uma seleção definida em uma visualização para _filtrar_ o conteúdo de outra visualização.

Vamos construir uma coleção de histogramas para o conjunto de dados voos (_flights_): atraso (_delay_) de chegada (quão cedo ou tarde um vôo chega, em minutos), distância (_distance_) voada (em milhas), e tempo (_time_) de partida (hora do dia). Usaremos o operador `repeat` para criar os histogramas, e adicionaremos uma seleção `interval` para o eixo `x` com seleções via interseção.

Em particular, cada histograma consistirá em duas camadas: uma camada de fundo cinzento e uma camada de primeiro plano azul, com a camada de primeiro plano filtrada pela nossa interseção de seleções. O resultado é uma interação de _filtragem cruzada_ entre os três gráficos!

_Arraste os intervalos das seleções nos gráficos abaixo. À medida que seleciona voos com atrasos de chegada mais longos ou mais curtos, como é que as distribuições de distância e tempo respondem?_

In [10]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 divisórias
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # eixo y sem título
)
  
alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # atraso de resposta > 3 horas
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionadas
).configure_view(
    stroke='transparent' # sem outline
)

C:\Users\elian\AppData\Local\Temp\ipykernel_19272\4166571633.py:15: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  hist.add_selection(brush).encode(color=alt.value('lightgrey')),


alt.RepeatChart(...)

_Ao fazer uma filtragem cruzada, pode-se observar que os vôos atrasados têm maior probabilidade de partir a horas mais tardias. Este fenômeno é frequentemente familiar aos passageiros: um atraso pode propagar-se ao longo do dia, afetando as viagens subsequentes nesse avião. Para ter maior probabilidade de chegar na hora, reserve um vôo cedo!_

A combinação de múltiplas visualizações e seleções interativas pode permitir formas valiosas de raciocínio multidimensional, transformando até histogramas básicos em poderosos dispositivos de entrada para formular perguntas sobre um conjunto de dados!